In [1]:
import numpy as np
#from lhereader import LHEReader

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

### extract info for LQ analysis

## SIGNAL LQ up

#### LQ up 800GeV

In [2]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-800GeV/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-800GeV/'


ev = []



for iev in range(1,176):
    
    inputevents = LHCO_reader.Events(f_name=Folder+'LQ_U_800-'+str(iev)+"/tag_1_delphes_events.lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

/home/andres/anaconda3/envs/py2/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


('run: ', 1, '   # events ok so far: ', 2487)
('run: ', 2, '   # events ok so far: ', 4898)
('run: ', 3, '   # events ok so far: ', 7322)
('run: ', 4, '   # events ok so far: ', 9784)
('run: ', 5, '   # events ok so far: ', 12295)
('run: ', 6, '   # events ok so far: ', 14816)
('run: ', 7, '   # events ok so far: ', 17303)
('run: ', 8, '   # events ok so far: ', 19688)
('run: ', 9, '   # events ok so far: ', 22195)
('run: ', 10, '   # events ok so far: ', 24667)
('run: ', 11, '   # events ok so far: ', 27032)
('run: ', 12, '   # events ok so far: ', 29487)
('run: ', 13, '   # events ok so far: ', 31950)
('run: ', 14, '   # events ok so far: ', 34312)
('run: ', 15, '   # events ok so far: ', 36832)
('run: ', 16, '   # events ok so far: ', 39156)
('run: ', 17, '   # events ok so far: ', 41594)
('run: ', 18, '   # events ok so far: ', 43976)
('run: ', 19, '   # events ok so far: ', 46391)
('run: ', 20, '   # events ok so far: ', 48869)
('run: ', 21, '   # events ok so far: ', 51245)
('run

('run: ', 168, '   # events ok so far: ', 407744)
('run: ', 169, '   # events ok so far: ', 410168)
('run: ', 170, '   # events ok so far: ', 412528)
('run: ', 171, '   # events ok so far: ', 414924)
('run: ', 172, '   # events ok so far: ', 417319)
('run: ', 173, '   # events ok so far: ', 419721)
('run: ', 174, '   # events ok so far: ', 422085)
('run: ', 175, '   # events ok so far: ', 424488)
 
('Total pasaron los cortes: ', 424488)


In [ ]:
# total LQ up 800GeV     424488

In [24]:
2487*176

437712

#### LQ up 1000GeV

In [17]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-1000GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-1000GeV/'


ev = []



for iev in range(1,101):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2648)
('run: ', 2, '   # events ok so far: ', 5330)
('run: ', 3, '   # events ok so far: ', 7978)
('run: ', 4, '   # events ok so far: ', 10647)
('run: ', 5, '   # events ok so far: ', 13392)
('run: ', 6, '   # events ok so far: ', 16119)
('run: ', 7, '   # events ok so far: ', 18747)
('run: ', 8, '   # events ok so far: ', 21409)
('run: ', 9, '   # events ok so far: ', 24161)
('run: ', 10, '   # events ok so far: ', 26786)
('run: ', 11, '   # events ok so far: ', 29473)
('run: ', 12, '   # events ok so far: ', 32133)
('run: ', 13, '   # events ok so far: ', 34773)
('run: ', 14, '   # events ok so far: ', 37517)
('run: ', 15, '   # events ok so far: ', 40194)
('run: ', 16, '   # events ok so far: ', 42834)
('run: ', 17, '   # events ok so far: ', 45520)
('run: ', 18, '   # events ok so far: ', 48197)
('run: ', 19, '   # events ok so far: ', 50912)
('run: ', 20, '   # events ok so far: ', 53663)
('run: ', 21, '   # events ok so far: ', 56367)
('ru

In [ ]:
# total LQ up 1000GeV     267452

In [25]:
2648*100

264800

#### LQ up 1200GeV

In [18]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-1200GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-1200GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2768)
('run: ', 2, '   # events ok so far: ', 5472)
('run: ', 3, '   # events ok so far: ', 8205)
('run: ', 4, '   # events ok so far: ', 10874)
('run: ', 5, '   # events ok so far: ', 13524)
('run: ', 6, '   # events ok so far: ', 16135)
('run: ', 7, '   # events ok so far: ', 18779)
('run: ', 8, '   # events ok so far: ', 21474)
('run: ', 9, '   # events ok so far: ', 24203)
('run: ', 10, '   # events ok so far: ', 26824)
('run: ', 11, '   # events ok so far: ', 29471)
('run: ', 12, '   # events ok so far: ', 32096)
('run: ', 13, '   # events ok so far: ', 34821)
('run: ', 14, '   # events ok so far: ', 37507)
('run: ', 15, '   # events ok so far: ', 40194)
('run: ', 16, '   # events ok so far: ', 42905)
('run: ', 17, '   # events ok so far: ', 45539)
('run: ', 18, '   # events ok so far: ', 48193)
('run: ', 19, '   # events ok so far: ', 50865)
('run: ', 20, '   # events ok so far: ', 53623)
('run: ', 21, '   # events ok so far: ', 56331)
('ru

('run: ', 168, '   # events ok so far: ', 448814)
('run: ', 169, '   # events ok so far: ', 451401)
('run: ', 170, '   # events ok so far: ', 454134)
('run: ', 171, '   # events ok so far: ', 456757)
('run: ', 172, '   # events ok so far: ', 459438)
('run: ', 173, '   # events ok so far: ', 462107)
('run: ', 174, '   # events ok so far: ', 464763)
('run: ', 175, '   # events ok so far: ', 467359)
('run: ', 176, '   # events ok so far: ', 470016)
('run: ', 177, '   # events ok so far: ', 472695)
('run: ', 178, '   # events ok so far: ', 475412)
('run: ', 179, '   # events ok so far: ', 478082)
('run: ', 180, '   # events ok so far: ', 480810)
('run: ', 181, '   # events ok so far: ', 483485)
('run: ', 182, '   # events ok so far: ', 486130)
('run: ', 183, '   # events ok so far: ', 488869)
('run: ', 184, '   # events ok so far: ', 491549)
('run: ', 185, '   # events ok so far: ', 494243)
('run: ', 186, '   # events ok so far: ', 496900)
('run: ', 187, '   # events ok so far: ', 499549)


In [ ]:
# total LQ up 1200GeV     534247

In [26]:
2768*200

553600

#### LQ up 1400GeV

In [19]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-1400GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-1400GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2530)
('run: ', 2, '   # events ok so far: ', 5078)
('run: ', 3, '   # events ok so far: ', 7556)
('run: ', 4, '   # events ok so far: ', 10052)
('run: ', 5, '   # events ok so far: ', 12633)
('run: ', 6, '   # events ok so far: ', 15170)
('run: ', 7, '   # events ok so far: ', 17744)
('run: ', 8, '   # events ok so far: ', 20331)
('run: ', 9, '   # events ok so far: ', 22904)


/home/andres/anaconda3/envs/py2/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:1484: UserWarning: Adding empty event
  warnings.warn("Adding empty event")


('run: ', 10, '   # events ok so far: ', 22904)
('run: ', 11, '   # events ok so far: ', 22904)
('run: ', 12, '   # events ok so far: ', 25454)
('run: ', 13, '   # events ok so far: ', 28028)
('run: ', 14, '   # events ok so far: ', 30550)
('run: ', 15, '   # events ok so far: ', 33109)
('run: ', 16, '   # events ok so far: ', 35699)
('run: ', 17, '   # events ok so far: ', 38240)
('run: ', 18, '   # events ok so far: ', 40801)
('run: ', 19, '   # events ok so far: ', 43346)
('run: ', 20, '   # events ok so far: ', 45976)
('run: ', 21, '   # events ok so far: ', 48514)
('run: ', 22, '   # events ok so far: ', 51120)
('run: ', 23, '   # events ok so far: ', 53708)
('run: ', 24, '   # events ok so far: ', 56182)
('run: ', 25, '   # events ok so far: ', 58769)
('run: ', 26, '   # events ok so far: ', 61267)
('run: ', 27, '   # events ok so far: ', 63910)
('run: ', 28, '   # events ok so far: ', 66458)
('run: ', 29, '   # events ok so far: ', 69028)
('run: ', 30, '   # events ok so far: ',

('run: ', 177, '   # events ok so far: ', 447448)
('run: ', 178, '   # events ok so far: ', 449963)
('run: ', 179, '   # events ok so far: ', 452457)
('run: ', 180, '   # events ok so far: ', 455008)
('run: ', 181, '   # events ok so far: ', 457666)
('run: ', 182, '   # events ok so far: ', 460139)
('run: ', 183, '   # events ok so far: ', 462652)
('run: ', 184, '   # events ok so far: ', 465196)
('run: ', 185, '   # events ok so far: ', 467691)
('run: ', 186, '   # events ok so far: ', 470306)
('run: ', 187, '   # events ok so far: ', 472834)
('run: ', 188, '   # events ok so far: ', 475432)
('run: ', 189, '   # events ok so far: ', 478022)
('run: ', 190, '   # events ok so far: ', 480590)
('run: ', 191, '   # events ok so far: ', 483192)
('run: ', 192, '   # events ok so far: ', 485763)
('run: ', 193, '   # events ok so far: ', 488278)
('run: ', 194, '   # events ok so far: ', 490959)
('run: ', 195, '   # events ok so far: ', 493460)
('run: ', 196, '   # events ok so far: ', 495989)


In [ ]:
# total LQ up 1400GeV     506381

In [27]:
2530*200

506000

#### LQ up 1600GeV

In [20]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-1600GeV/Auto_Results_LHCO/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-1600GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+"run_0"+str(iev)+".lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2386)
('run: ', 2, '   # events ok so far: ', 4857)
('run: ', 3, '   # events ok so far: ', 7326)
('run: ', 4, '   # events ok so far: ', 9808)
('run: ', 5, '   # events ok so far: ', 12205)
('run: ', 6, '   # events ok so far: ', 14671)
('run: ', 7, '   # events ok so far: ', 17087)
('run: ', 8, '   # events ok so far: ', 19538)
('run: ', 9, '   # events ok so far: ', 21954)
('run: ', 10, '   # events ok so far: ', 24336)
('run: ', 11, '   # events ok so far: ', 26792)
('run: ', 12, '   # events ok so far: ', 29200)
('run: ', 13, '   # events ok so far: ', 31567)
('run: ', 14, '   # events ok so far: ', 34008)
('run: ', 15, '   # events ok so far: ', 36444)
('run: ', 16, '   # events ok so far: ', 38882)
('run: ', 17, '   # events ok so far: ', 41365)
('run: ', 18, '   # events ok so far: ', 43750)
('run: ', 19, '   # events ok so far: ', 46241)
('run: ', 20, '   # events ok so far: ', 48635)
('run: ', 21, '   # events ok so far: ', 51028)
('run

('run: ', 168, '   # events ok so far: ', 405448)
('run: ', 169, '   # events ok so far: ', 407767)
('run: ', 170, '   # events ok so far: ', 410139)
('run: ', 171, '   # events ok so far: ', 412560)
('run: ', 172, '   # events ok so far: ', 414937)
('run: ', 173, '   # events ok so far: ', 417291)
('run: ', 174, '   # events ok so far: ', 419704)
('run: ', 175, '   # events ok so far: ', 422075)
('run: ', 176, '   # events ok so far: ', 424465)
('run: ', 177, '   # events ok so far: ', 426879)
('run: ', 178, '   # events ok so far: ', 429412)
('run: ', 179, '   # events ok so far: ', 431752)
('run: ', 180, '   # events ok so far: ', 434124)
('run: ', 181, '   # events ok so far: ', 436587)
('run: ', 182, '   # events ok so far: ', 438991)
('run: ', 183, '   # events ok so far: ', 441428)
('run: ', 184, '   # events ok so far: ', 443771)
('run: ', 185, '   # events ok so far: ', 446078)
('run: ', 186, '   # events ok so far: ', 448423)
('run: ', 187, '   # events ok so far: ', 450777)


In [6]:
# total LQ up 1600GeV     481827

In [28]:
2386*200

477200

#### LQ up 1800GeV

In [22]:
Folder = '/media/andres/Elements/Leptoquarks-v3/signals/LQup-1800GeV/'
datFolder = '/media/andres/Elements/Leptoquarks-v3/signals/DATA/dats-LQup-1800GeV/'


ev = []



for iev in range(1,201):
    
    inputevents = LHCO_reader.Events(f_name=Folder+'lqu18-'+str(iev)+"/tag_1_delphes_events.lhco")

    for i in range(len(inputevents)):

        num_es = 0
        num_mus = 0
        num_taus = 0
        num_jets = 0
        num_btag = 0
        index_tau = 0
        sT = 0
        mTtau = 0
        mTb1b2 = 0
        HT = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
        
        b_pt = []
        b_eta = []
        b_phi = []
        
        # to be considered e
        if inputevents[i].number()["electron"] > 0:
            for ll in range(inputevents[i].number()["electron"]):
                if inputevents[i]["electron"][ll]["PT"] > 10 and -2.47 < inputevents[i]["electron"][ll]["eta"] < 2.47:
                    num_es += 1
                    
        # to be considered mu
        if inputevents[i].number()["muon"] > 0:
            for ll in range(inputevents[i].number()["muon"]):
                if inputevents[i]["muon"][ll]["PT"] > 10 and -2.7 < inputevents[i]["muon"][ll]["eta"] < 2.7:
                    num_mus += 1
                    
        # to be considered tau
        if inputevents[i].number()["tau"] > 0:
            for ll in range(inputevents[i].number()["tau"]):
                if inputevents[i]["tau"][ll]["PT"] > 20 and (-2.5 < inputevents[i]["tau"][ll]["eta"] < -1.52 or -1.37 < inputevents[i]["tau"][ll]["eta"] < 1.37 or 1.52 < inputevents[i]["tau"][ll]["eta"] < 2.5):
                    num_taus += 1
                    index_tau = ll
                    
        # conditions A: 0 electrons, 0 muons, 1 tau and also EXTRA: require that the id tau has pT > 50 GeV
        #if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 50:
        if num_es == 0 and num_mus == 0 and num_taus == 1 and inputevents[i]["tau"][index_tau]["PT"] > 20:

            # MET
            if inputevents[i]["MET"][0]["PT"] > 280:

                # at least 2 jets
                if inputevents[i].number()["jet"] > 1:
                    
                    # transverse mass of leading tau (that was id as tau)
                    mTtau = (2 * inputevents[i]["tau"][index_tau]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["tau"][index_tau]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5

                    # condition B: mTtau > 150 GeV
                    #if mTtau > 150:
                    if mTtau > 0:
                        
                        # compute sT using the tau that was id as tau
                        sT = inputevents[i]["tau"][index_tau]["PT"]
                        
                        # add to sT the 2 leading jets (id as jets)
                        for k in range(inputevents[i].number()["jet"]):
                            if -2.8 < inputevents[i]["jet"][k]["eta"] < 2.8 and inputevents[i]["jet"][k]["PT"] > 20:
                                num_jets += 1
                                HT += inputevents[i]["jet"][k]["PT"]
                                if num_jets < 3:
                                    sT = sT + inputevents[i]["jet"][k]["PT"]

                        # check # of b-jets (id as b-tag, tighter request than jets)
                        for j in range(inputevents[i].number()["jet"]):
                            if inputevents[i]["jet"][j]["btag"] == 1 and inputevents[i]["jet"][j]["PT"] > 20 and -2.5 < inputevents[i]["jet"][j]["eta"] < 2.5:
                                num_btag += 1
                                if num_btag < 3:
                                    mTb1b2 += (2 * inputevents[i]["jet"][j]["PT"] * inputevents[i]["MET"][0]["PT"] * (1.0- np.cos(inputevents[i]["jet"][j]["phi"]-inputevents[i]["MET"][0]["phi"])) )**0.5
                                        
                                    b_pt.append(inputevents[i]["jet"][j]["PT"])
                                    b_eta.append(inputevents[i]["jet"][j]["eta"])
                                    b_phi.append(inputevents[i]["jet"][j]["phi"])


                        # conditions C: 2 or more jets, 2 or more b-jets, sT > 600 GeV and mTb1b2 > 700 GeV
                        #if num_jets > 1 and num_btag > 1 and sT > 600 and mTb1b2 > 700:
                        if num_jets > 1 and num_btag > 1 and sT > 0 and mTb1b2 > 0:

                            ev.append(i)
                            
                            # SAVE: 
                            # hadronic info
                            with open(datFolder+'hadinfo.dat', 'a') as f:
                                f.write('%f %d %d\n' % (HT, num_jets, num_btag))
                            
                            # b info:
                            with open(datFolder+'b1.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[0], b_eta[0], b_phi[0]))
                            with open(datFolder+'b2.dat', 'a') as f:
                                f.write('%f %f %f\n' % (b_pt[1], b_eta[1], b_phi[1]))
                                
                            # tau
                            with open(datFolder+'tau.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["tau"][0]["PT"],inputevents[i]["tau"][0]["eta"],inputevents[i]["tau"][0]["phi"]))
                            
                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder+'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))
                            

 
    print('run: ',iev, '   # events ok so far: ' , len(ev))
print(' ')
print('Total pasaron los cortes: ', len(ev))

('run: ', 1, '   # events ok so far: ', 2157)
('run: ', 2, '   # events ok so far: ', 4358)
('run: ', 3, '   # events ok so far: ', 6632)
('run: ', 4, '   # events ok so far: ', 8888)
('run: ', 5, '   # events ok so far: ', 11112)
('run: ', 6, '   # events ok so far: ', 13332)
('run: ', 7, '   # events ok so far: ', 15691)
('run: ', 8, '   # events ok so far: ', 17894)
('run: ', 9, '   # events ok so far: ', 20143)
('run: ', 10, '   # events ok so far: ', 22339)
('run: ', 11, '   # events ok so far: ', 24527)
('run: ', 12, '   # events ok so far: ', 26787)
('run: ', 13, '   # events ok so far: ', 29010)
('run: ', 14, '   # events ok so far: ', 31225)
('run: ', 15, '   # events ok so far: ', 33512)
('run: ', 16, '   # events ok so far: ', 35749)
('run: ', 17, '   # events ok so far: ', 38028)
('run: ', 18, '   # events ok so far: ', 40178)
('run: ', 19, '   # events ok so far: ', 42457)
('run: ', 20, '   # events ok so far: ', 44730)
('run: ', 21, '   # events ok so far: ', 46910)
('run

('run: ', 168, '   # events ok so far: ', 278996)
('run: ', 169, '   # events ok so far: ', 279755)
('run: ', 170, '   # events ok so far: ', 280536)
('run: ', 171, '   # events ok so far: ', 281263)
('run: ', 172, '   # events ok so far: ', 282038)
('run: ', 173, '   # events ok so far: ', 282828)
('run: ', 174, '   # events ok so far: ', 283555)
('run: ', 175, '   # events ok so far: ', 284297)
('run: ', 176, '   # events ok so far: ', 285041)
('run: ', 177, '   # events ok so far: ', 285853)
('run: ', 178, '   # events ok so far: ', 286568)
('run: ', 179, '   # events ok so far: ', 287331)
('run: ', 180, '   # events ok so far: ', 288050)
('run: ', 181, '   # events ok so far: ', 288739)
('run: ', 182, '   # events ok so far: ', 289428)
('run: ', 183, '   # events ok so far: ', 290162)
('run: ', 184, '   # events ok so far: ', 290874)
('run: ', 185, '   # events ok so far: ', 291523)
('run: ', 186, '   # events ok so far: ', 292209)
('run: ', 187, '   # events ok so far: ', 292928)


In [ ]:
# total LQ up 1800GeV     301687

In [29]:
2157*200

431400